In [1]:
from FX.FX import SQLAnaforFX, OHLCanalyzer, SMAanalyzer

Using TensorFlow backend.


# データベースの初期化とデータの挿入

In [2]:
year_month = "201705"
sql = SQLAnaforFX(year_month=year_month, recreate=True) # 本当に初期化する場合はTrueにする
sql.addFXRecordsFromFolder("../../data/{}/".format(year_month))
res = sql.toDataFrame(tblname="main")

Start insertion of the data in folder: ../../data/201705/
Finished. Elapsed time: 23.69 sec.


# OHLCの計算と挿入

In [3]:
sql.initialize_ana_tables(recreate=True)
ana = OHLCanalyzer(res)
for ii in range(len(ana._mins)):
    ana.calc_ohlc(ana._mins[ii])
    asks, bids = ana.toDataFrame()
    sql.addOHLCRecordFromDataFrame(asks, ana._mins[ii], "ask")
    sql.addOHLCRecordFromDataFrame(bids, ana._mins[ii], "bid")

# SMAの計算と挿入

In [4]:
ana = SMAanalyzer()

for bid_or_ask in ["ask", "bid"]:
    for minute in sql._mins:
        df = sql.AnaToDataFrame(minute=minute, bid_or_ask=bid_or_ask, colselect=["close"])
        ana.calc_sma(df)
        df = ana.toDataFrame()
        sql.addSMARecordFromDataFrame(df, minute=minute, bid_or_ask=bid_or_ask)

In [5]:
res = sql.toDataFrame(tblname="main")
res.head(12), len(res)

(                          dateval     ask     bid
 2017-05-01 07:01:30  2.017050e+13  111.37  111.36
 2017-05-01 07:01:35  2.017050e+13  111.38  111.37
 2017-05-01 07:01:40  2.017050e+13  111.38  111.37
 2017-05-01 07:01:45  2.017050e+13  111.38  111.37
 2017-05-01 07:01:51  2.017050e+13  111.38  111.37
 2017-05-01 07:01:56  2.017050e+13  111.38  111.37
 2017-05-01 07:02:01  2.017050e+13  111.38  111.37
 2017-05-01 07:02:06  2.017050e+13  111.39  111.38
 2017-05-01 07:02:11  2.017050e+13  111.39  111.38
 2017-05-01 07:02:16  2.017050e+13  111.38  111.37
 2017-05-01 07:02:21  2.017050e+13  111.39  111.38
 2017-05-01 07:02:26  2.017050e+13  111.39  111.38, 369129)

In [6]:
res = sql.toDataFrame(tblname="ask01min")
res.head(12), len(res)

(                       open    high     low   close       sma07       sma13  \
 2017-05-01 07:02:30  111.37  111.39  111.37  111.39  111.390000  111.390000   
 2017-05-01 07:03:30  111.39  111.40  111.38  111.39  111.390000  111.390000   
 2017-05-01 07:04:30  111.39  111.41  111.39  111.39  111.390000  111.390000   
 2017-05-01 07:05:30  111.38  111.39  111.35  111.35  111.380000  111.380000   
 2017-05-01 07:06:30  111.35  111.35  111.30  111.30  111.364000  111.364000   
 2017-05-01 07:07:30  111.31  111.31  111.30  111.30  111.353333  111.353333   
 2017-05-01 07:08:30  111.30  111.32  111.30  111.31  111.347143  111.347143   
 2017-05-01 07:09:30  111.31  111.33  111.30  111.33  111.338571  111.345000   
 2017-05-01 07:10:30  111.33  111.33  111.33  111.33  111.330000  111.343333   
 2017-05-01 07:11:30  111.33  111.33  111.32  111.33  111.321429  111.342000   
 2017-05-01 07:12:30  111.33  111.33  111.30  111.30  111.314286  111.338182   
 2017-05-01 07:13:30  111.30  111.31  11

# 微分の導入